In [12]:
# importing dependencies
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb

### Reading Data files Train and Test

In [ ]:
#drop columns CASEID, YEAR,DISYR from train data
data=pd.read_csv("HackTrain.csv")
data=data.drop(['CASEID','YEAR','DISYR'],axis=1)

#drop columns CASEID, DISYR from test data
test=pd.read_csv("HackTest.csv")
test = test.drop(['CASEID','DISYR'], axis=1)

### Data Preprocessing

In [ ]:
# replace null values with median in train and test data
data=data.replace(-9.0,numpy.NaN)
data=data.fillna(data.median())

test=test.replace(-9.0,numpy.NaN)
test=test.fillna(test.median())

#select target feature for classification
y_classification=data['REASON']
X_classification=data.drop(['REASON','LOS'],axis=1)

# select target feature for regression
y_regression=data['LOS']
X_regression=data.drop(['REASON','LOS'],axis=1)

# normalizing data
scaler = StandardScaler()
X_classification_scaled=scaler.fit_transform(X_classification)
X_regression_scaled=scaler.fit_transform(X_regression)
test=scaler.fit_transform(test)








### Train the classification model

In [13]:
#split data into train and test
X_train_classification, X_test_classification, y_train, y_test = train_test_split(X_classification_scaled, y_classification, test_size=0.33, random_state=42,stratify=y_classification)

fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test_classification,y_test)],
            'eval_names': ['valid'],            
            'verbose': 100,
            'categorical_feature': 'auto'}

clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=800)
opt_parameters = {'colsample_bytree': 0.9234, 'min_child_samples': 399, 'min_child_weight': 0.1, 'num_leaves': 120, 'reg_alpha': 2, 'reg_lambda': 5, 'subsample': 0.855}

clf_final = lgb.LGBMClassifier(**clf.get_params())
#choosing optimal parameters
clf_final.set_params(**opt_parameters)
clf_final.fit(X_classification_scaled, y_classification, **fit_params)
#predict test data values using trained model
y_pred_classification=clf_final.predict(test)

Training until validation scores don't improve for 30 rounds.
[100]	valid's auc: 0.805955
[200]	valid's auc: 0.813876
[300]	valid's auc: 0.81943
[400]	valid's auc: 0.823801
[500]	valid's auc: 0.827405
[600]	valid's auc: 0.830628
[700]	valid's auc: 0.833786
[800]	valid's auc: 0.836649
[900]	valid's auc: 0.839422
Did not meet early stopping. Best iteration is:
[900]	valid's auc: 0.839422


D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Train the regression model

In [14]:
d_train = lgb.Dataset(X_regression_scaled, y_regression)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 100,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

gbm = lgb.train(params,
                d_train,
                num_boost_round=1000)
#predict test data values using trained model
y_pred_regression=gbm.predict(test)

### Export test data file

In [16]:
predictions=pd.DataFrame()
test=pd.read_csv("HackTest.csv")
predictions['ID']=test['CASEID']
predictions['LOS']=y_pred_regression
predictions['REASON']=y_pred_classification
predictions.to_csv('predictions320.csv',index=False,header=False)